In [7]:
import redcap
import pandas as pd
data = {
    'token': '085520489DF98911614E03FF05D58EA8',
    'content': 'project',
    'format': 'csv',
    'returnFormat': 'csv'
}

proj = redcap.Project('https://ruff.mrc.gm:8443/redcap/api/', data['token'])

In [8]:
crf_name = 'ironsupplement'

# Use the export_records() function to export the specified CRF
crf_data = proj.export_records(forms=[crf_name])

crf_data = pd.DataFrame(crf_data)
counts = (
    (crf_data['iron_supferrouspri_q2'] == '1') |
    (crf_data['ip_administer'] == '1')
).groupby(crf_data['cons_recordid_q1']).sum().rename('Total_ips')

counts2 = (
    (crf_data['iron_supferrouspri_q2'] == '1') &
    (crf_data['ip_administer'] == '1')
).groupby(crf_data['cons_recordid_q1']).sum().rename('Total_ips_both')

total_counts = counts + counts2
total_counts.name = 'Total_ips_combined'

crf_data = pd.merge(crf_data, total_counts, on=['cons_recordid_q1'], how='inner')
crf_data['total_IP_issued'] = 84
crf_data['balance'] = crf_data['total_IP_issued'] - crf_data['Total_ips_combined']

# Calculate the count of each instrument
instrument_counts = crf_data['redcap_repeat_instrument'].value_counts()

# Find the instrument with the maximum count
max_count_instrument = instrument_counts.idxmax()

# Group by the highest instrument count
grouped_data = crf_data.groupby('redcap_repeat_instrument').filter(lambda x: x['redcap_repeat_instrument'].value_counts().max() == instrument_counts.max())

# Filter out cons_recordid_q1 values starting with "IDV"
grouped_data = grouped_data[~grouped_data['cons_recordid_q1'].str.startswith('IDV')]

# Save the grouped data to a CSV file
grouped_data.drop_duplicates('cons_recordid_q1').to_csv('monitors_ip_log.csv',index=False)

In [9]:
ran=proj.export_records(forms=['randomization'])
ran=pd.DataFrame(ran)

ran=ran[
    (ran['randomization_complete']=='2')
]
ran=ran[['cons_recordid_q1','rand_treatgroup_q3']]

In [10]:
mmonitors_ip_lo=pd.read_csv('monitors_ip_log.csv')
mmonitors_ip_lo=pd.DataFrame(mmonitors_ip_lo)



In [11]:
mmonitors_ip_lo=pd.merge(mmonitors_ip_lo,ran,on='cons_recordid_q1',how='inner')

mmonitors_ip_lo.to_csv('monitors_ip_log_1.csv',index=False)

In [12]:
v12_sysmex=proj.export_records(forms=['sysmexresult'])
v12_sysmex=pd.DataFrame(v12_sysmex)
v12_sysmex=v12_sysmex[['cons_recordid_q1', 'redcap_event_name', 'redcap_repeat_instrument',
       'redcap_repeat_instance','sysmexresult_complete']]


In [13]:
filter_data=v12_sysmex[
    (v12_sysmex['redcap_event_name']=='v12_arm_1')&
    (v12_sysmex['sysmexresult_complete']=='2')
]

filter_data.to_csv('completed_fellow_up_IDeA.csv',index=False)

In [14]:
withdraw_data=proj.export_records(forms=['withdrawal'])
withdraw_data=pd.DataFrame(withdraw_data)

withdraw_data=withdraw_data[
    (withdraw_data['withdrawal_complete']=='2')|
    (withdraw_data['withdrawal_complete']=='1')
]

In [15]:

# Find record IDs not present in filter_data
ip_log = grouped_data[~grouped_data['cons_recordid_q1'].isin(filter_data['cons_recordid_q1'])]

# Filter out records from withdraw_data that are in ip_log
current_IP = ip_log[~ip_log['cons_recordid_q1'].isin(withdraw_data['cons_recordid_q1'])]
#current_IP.to_csv('ip_log.csv', index=False)
current_IP.drop_duplicates('cons_recordid_q1').to_csv('ip_log.csv', index=False)
# Save ip_log to a CSV file



In [16]:
grouped_data=grouped_data[['cons_recordid_q1','Total_ips_combined',
       'total_IP_issued', 'balance']]

In [17]:

ENDLINE_RECORD=pd.read_csv('Merghed_Queries.csv')
ENDLINE_RECORD=pd.DataFrame(ENDLINE_RECORD)

In [18]:
ENDLINE_RECORD_merg=pd.merge(grouped_data,ENDLINE_RECORD,on='cons_recordid_q1',how='inner')
ENDLINE_RECORD_merg.drop_duplicates('cons_recordid_q1').to_csv('Merghed_Queries.csv',index=False)

In [19]:
#ran=proj.export_records(forms=['randomization'])
#ran=pd.DataFrame(ran)


In [20]:
#ran_column=ran[['cons_recordid_q1','rand_treatgroup_q3']]
#ran_column=pd.DataFrame(ran_column)

In [21]:
#mergh_data=pd.merge(grouped_data,ran_column,on='cons_recordid_q1',how='inner')
#mergh_data.to_csv('mergh_dat.csv',index=False)

In [22]:
#reconcile_data=pd.read_excel('Book3.xlsx')
#reconcile_data=pd.DataFrame(reconcile_data)

#reconcile_data=pd.merge(reconcile_data,ran_column,on='cons_recordid_q1',how='inner')
#reconcile_data.to_csv('tobe_reconcile.csv',index=False)

In [23]:
inner_join_sysmex_iron=pd.merge(current_IP, filter_data,on='cons_recordid_q1', how='inner')